In [1]:
import pyomo.environ as pyo

#Data
# Costs for each edge
cost = {
    (1, 3): 7,   (1, 8): 8,   (1, 4): 7,
    (2, 4): 7,   (2, 8): 4,   (3, 4): 0,
    (3, 5): 25,  (3, 6): 6,   (3, 7): 17,
    (4, 3): 0,   (4, 6): 8,   (4, 7): 5
}

# Capacities for each edge (∞ is represented by float('inf'))
capacity = {
    (1, 3): float('inf'),  (1, 8): float('inf'),  (1, 4): float('inf'),
    (2, 4): float('inf'),  (2, 8): float('inf'),  (3, 4): 25,
    (3, 5): float('inf'),  (3, 6): float('inf'),  (3, 7): float('inf'),
    (4, 3): 25,            (4, 6): float('inf'),  (4, 7): float('inf')
}

# Demand at each node
demand = {
    1: -1000,  # Wisconsin (Supply 1000)
    2: -1000,  # Alabama (Supply 1000)
    3: 0,      # Memphis
    4: 0,      # Pittsburgh
    5: 450,    # Fresno (Demand 450)
    6: 500,    # Peoria (Demand 500)
    7: 610,    # Newark (Demand 610)
    8: 440     # Unspecified node (Demand 440)
}

# Create a model
model = pyo.ConcreteModel()

# Define sets
model.nodes = pyo.Set(initialize=demand.keys())
model.edges = pyo.Set(initialize=cost.keys())

# Define variables
model.flow = pyo.Var(model.edges, domain=pyo.NonNegativeReals)

# Define objective function
def obj_rule(model):
    return sum(cost[i, j] * model.flow[i, j] for i, j in model.edges)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Define constraints
def flow_bal_rule(model, node):
    in_edges = [(i, j) for i, j in model.edges if j == node]
    out_edges = [(i, j) for i, j in model.edges if i == node]
    return sum(model.flow[i, j] for i, j in in_edges) - sum(model.flow[i, j] for i, j in out_edges) == demand[node]
model.flow_bal = pyo.Constraint(model.nodes, rule=flow_bal_rule)

def capacity_rule(model, i, j):
    return model.flow[i, j] <= capacity[i, j]
model.capacity = pyo.Constraint(model.edges, rule=capacity_rule)

# Solve the model
opt = pyo.SolverFactory('gurobi')
opt.options['Method'] = 0
opt.solve(model, tee=True)

# Display the results
print('\nFlow on each edge:')
for i, j in model.edges:
    print(f'Flow from {i} to {j}: {model.flow[i, j].value}')

print('\nObjective value:', model.obj())


Read LP format model from file /tmp/tmprpjoww1i.pyomo.lp
Reading time = 0.00 seconds
x1: 10 rows, 12 columns, 26 nonzeros
Set parameter Method to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
Method  0

Optimize a model with 10 rows, 12 columns and 26 nonzeros
Model fingerprint: 0x0c7ff197
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+03]
Presolve removed 10 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9980000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.99800000

In [2]:

# Solve the model
opt.options['Method'] = 2
opt.options['Crossover'] = 0
opt.solve(model, tee=True)

# Display the results
print('\nFlow on each edge:')
for i, j in model.edges:
    print(f'Flow from {i} to {j}: {model.flow[i, j].value}')

print('\nObjective value:', model.obj())

Read LP format model from file /tmp/tmp5o68qlku.pyomo.lp
Reading time = 0.00 seconds
x1: 10 rows, 12 columns, 26 nonzeros
Set parameter Method to value 2
Set parameter Crossover to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
Method  2
Crossover  0

Optimize a model with 10 rows, 12 columns and 26 nonzeros
Model fingerprint: 0x0c7ff197
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+03]
Presolve removed 10 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective 2.99800000e+04

Flow on each edge:
Flow from 1 to 3: 925.0
Flow from 1 to 8: 